In [1]:
import os

In [2]:
%pwd

'd:\\MachineLearning\\Nlp project1\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\MachineLearning\\Nlp project1\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen =True)
class DataTransformationConfig:
  root_dir: Path
  data_path: Path
  tokenizer_name: Path

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_data_transformation(self) -> DataTransformationConfig:
      config = self.config.data_transformation
      
      
      data_transformation_config = DataTransformationConfig(
        root_dir = config.root_dir,
        data_path = config.data_path,
        tokenizer_name = config.tokenizer_name
      )
      
      return data_transformation_config

In [20]:
import os
from src.textSummarizer.logging import logger
from datasets import load_dataset,load_from_disk
from transformers import AutoTokenizer

import os

In [21]:


class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    def convert_examples_to_feature(self, example_batch):
        input_encodings = self.tokenizer(
            example_batch['dialogue'],
            max_length=1024,
            truncation=True,
            padding="max_length",  # Add padding
        )

        target_encodings = self.tokenizer(
            example_batch['summary'],
            max_length=128,
            truncation=True,
            padding="max_length",  # Add padding
        )

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids'],
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(
            self.convert_examples_to_feature, batched=True
        )
        dataset_samsum_pt.save_to_disk(
            os.path.join(self.config.root_dir, "samsum_dataset")
        )


In [22]:
try:
  config = ConfigurationManager()
  
  data_transformation_config = config.get_data_transformation()
  
  data_transformation = DataTransformation(config = data_transformation_config)
  data_transformation.convert() 
  
except Exception as e:
  raise e
  

[2023-08-21 22:35:31,492: INFO: common: yaml file: config\configuration.yaml loaded successfully]
[2023-08-21 22:35:31,495: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-21 22:35:31,497: INFO: common: created directory at : artifacts]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 32721.44 examples/s]


: 